#instalação

In [ ]:
%pip install -U pytube # instalando a biblioteca para baixar videos do youtube
%pip install -U openai-whisper # instalando a biblioteca para transcrição

# carregamentos das bibliotecas

In [2]:
import datetime # biblioteca para colocar os tempos na legenda
import os # biblioteca para gerenciar pastas do sistema
import whisper # biblioteca para fazer a transcrição
from pytube import YouTube # biblioteca para fazer downloads de vídeos do youtube

# Carregamento do modelo

## Modelos disponiveis:

https://github.com/openai/whisper#available-models-and-languages

lembre-se de carregar somente um modelo, coloquei dois como exemplo, deixe apenas o que vai utilizar

In [ ]:
model = whisper.load_model("tiny") # carregamento do modelo pequeno (não tão bom, mas muito rápido)
# model = whisper.load_model("large-v3") # carregamento do modelo (muito bom, mas não tão rápido)

# Definição das funções

## Função de download do youtube

In [ ]:
def download_youtube_audio(link): # função que faz download
    # print("Downloading youtube video") # feedback avisando que o vídeo está sendo baixado, só retirar o # no inicio da linha que ele roda
    video = YouTube(link) # construção do objeto do vídeo
    stream = video.streams.get_audio_only() # download do áudio
    # stream = video.streams.get_highest_resolution() # download do vídeo
    try:
        file_path = stream.download("yt_downloads") # tenta colocar os downloads nessa pasta
    except FileNotFoundError: # se a pasta não existir:
        os.mkdir("yt_downloads") # ele cria
        file_path = stream.download("yt_downloads") # e coloca o video lá
    # print("Downloaded successfully") # feedback avisando que deu tudo certo, só retirar o # no inicio da linha que ele roda
    return os.path.abspath(file_path) # fala onde está o vídeo

## Função de transcrição

In [ ]:
def convert_audio_to_srt(file_path): # função que converte o audio
    global model # pega o modelo que foi importado
    def format_srt_time(timestamp): # função que formata a diferença de tempo para srt
        td_object = str(datetime.timedelta(seconds=timestamp))
        if "." in td_object:
            td_object = td_object.replace('.', ',') # troca o ponto por vírgula
            td_object = td_object[:td_object.rindex(',') + 4] # pega só até a terceira casa decimal
        return td_object # retorna o tempo formatado

    def generate_srt(transcript): # função que gera o srt
        srt_text = "" # começa com um texto vazio
        for i, segment in enumerate(transcript): # para cada segmento na transcrição
            start_time = format_srt_time(segment['start']) # define o tempo inicial do trecho
            end_time = format_srt_time(segment['end']) # define o tempo final do trecho
            text = segment['text'] # define o texto
            srt_text += f"{i + 1}\n{start_time} --> {end_time}\n{text}\n\n" # coloca tudo junto nesse formato e no final coloca para a próxima linha
        return srt_text # retorna o texto final

    audio = whisper.load_audio(file_path) # carrega o audio
    result = model.transcribe(audio) # pega a transcrição a partir do audio
    new_file = file_path[:file_path.rindex(".")] # pega o caminho do arquivo e o nome dele, só remove a extensão
    srt_text = generate_srt(result['segments']) # gera o srt baseado em cada segmento por meio da função
    # print(srt_text) # exibe na tela a transcrição, só retirar o # no inicio da linha que ele roda
    with open(f"{new_file}.srt", "w") as file: # cria um arquivo com o nome do arquivo mp4 só que desta vez no formato srt
        file.write(srt_text) # escreve o conteúdo no arquivo

# baixando todos os vídeos do youtube na lista e gerando o srt

In [ ]:
links = ["https://www.youtube.com/watch?v=o0LjwAohJO8",
         "https://www.youtube.com/watch?v=zkI1TZOKSrM"
        ] # lista de vídeos de teste para download
for link in links: # loop para executar pegar cada link da lista de links
    download_youtube_audio(link) # função que realiza o download a partir do link que foi passado pelo loop

## define o caminho e também passa cada arquivo baixado do youtube para gerar o srt

In [ ]:
path = "/content/yt_downloads" # pasta que você quer transcrever
for file in os.listdir(path): # loop para pegar todos os arquivos na pasta
    convert_audio_to_srt(f"{path}/{file}") # passa cada arquivo na função que vai converter para srt

# Transcrevendo todos os arquivos em um diretório

In [ ]:
path = "/content" # pasta que você quer transcrever
for file in os.listdir(path): # loop para pegar todos os arquivos na pasta
    try: # tenta
        convert_audio_to_srt(f"{path}/{file}") # passa cada arquivo na função que vai converter para srt
        print(f"gerado srt para o seguinte arquivo:\n{file}") # feedback avisando que foi gerado o srt para o arquivo (só retirar o # no inicio da linha que ele roda)
    except: # se der algum erro
        print(f"não foi possível gerar o srt para o seguinte arquivo:\n{file}") # feedback avisando que não foi possível gerar o srt para o arquivo